# Hello there!

In [3]:
from core import lexical_features

lexical_features.get_lexical_features('/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp')

[0,
 0,
 -2.665032816018519,
 -3.530030253505124,
 -2.9704144655697013,
 0.15018315018315018,
 15]

In [20]:
from core.syntactic_features import get_syntactic_features

get_syntactic_features('/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp')

INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp" ... 


[-5.60947179518496, 4.0, 0, 0, 1]

In [8]:
from core.cpp_keywords_TF import count_cppkeywords_tf

count_cppkeywords_tf(['/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp'])
count_cppkeywords_tf.__doc__